In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r'C:\Users\LeachJ\PycharmProjects\osm_annual_report\07BE001_Daily_Flow_ts.csv')

In [3]:
df['Date'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']], errors='coerce')

In [4]:
df

,STATION_NUMBER,YEAR,MONTH,DAY,FLOW,FLAG,Date
0,07BE001,1913,5,1,268.0,NaN,1913-05-01
1,07BE001,1913,5,2,251.0,NaN,1913-05-02
2,07BE001,1913,5,3,242.0,NaN,1913-05-03
3,07BE001,1913,5,4,234.0,NaN,1913-05-04
4,07BE001,1913,5,5,225.0,NaN,1913-05-05
...,...,...,...,...,...,...,...
39687,07BE001,2021,12,27,95.9,B,2021-12-27
39688,07BE001,2021,12,28,95.2,B,2021-12-28
39689,07BE001,2021,12,29,94.4,B,2021-12-29
39690,07BE001,2021,12,30,93.8,B,2021-12-30


In [5]:
df_record = df.copy()

df_count = df_record.groupby(['MONTH', 'DAY'])['FLOW'].count()
df_count = df_count.reset_index(level=None, drop=False)

df_min = df_record.groupby(['MONTH', 'DAY'])['FLOW'].min()
df_min = df_min.reset_index(level=None, drop=False)

df_25 = df_record.groupby(['MONTH', 'DAY'])['FLOW'].quantile(0.25)
df_25 = df_25.reset_index(level=None, drop=False)

df_50 = df_record.groupby(['MONTH', 'DAY'])['FLOW'].quantile(0.50)
df_50 = df_50.reset_index(level=None, drop=False)

df_75 = df_record.groupby(['MONTH', 'DAY'])['FLOW'].quantile(0.75)
df_75 = df_75.reset_index(level=None, drop=False)

df_max = df_record.groupby(['MONTH', 'DAY'])['FLOW'].max()
df_max = df_max.reset_index(level=None, drop=False)

In [6]:
df = pd.DataFrame([df_min.MONTH, 
                   df_min.DAY,
                   df_count.FLOW,
                   df_min.FLOW,
                   df_25.FLOW,
                   df_50.FLOW,
                   df_75.FLOW,
                   df_max.FLOW]).transpose()
df.columns = ['MONTH', 'DAY', 'n', 'MIN', 'p_25', 'p_50', 'p_75', 'MAX']

In [7]:
df['YEAR'] = np.full((len(df.MAX), 1), 2021)

In [14]:
df['FLOW'] = df_record[df_record['YEAR'] == 2021]['FLOW'].values

In [15]:
df = df.drop(df[(df["MONTH"] == 2) & (df["DAY"] == 29)].index)

In [16]:
df.reset_index(level=None, drop=True, inplace=True, col_level=0)
df['DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']], errors='coerce')

In [17]:
df

,MONTH,DAY,n,MIN,p_25,p_50,p_75,MAX,YEAR,FLOW,DATE
0,1.0,1.0,88.0,51.0,81.499998,102.000000,122.00,170.0,2021,130.0,2021-01-01
1,1.0,2.0,88.0,51.0,80.275002,102.000000,120.25,173.0,2021,130.0,2021-01-02
2,1.0,3.0,88.0,50.5,78.424999,100.500000,119.00,182.0,2021,129.0,2021-01-03
3,1.0,4.0,88.0,50.0,77.699997,100.000000,119.50,180.0,2021,128.0,2021-01-04
4,1.0,5.0,88.0,51.0,76.799999,99.699997,116.75,191.0,2021,128.0,2021-01-05
...,...,...,...,...,...,...,...,...,...,...,...
360,12.0,27.0,88.0,47.0,78.200001,102.000000,125.25,213.0,2021,95.9,2021-12-27
361,12.0,28.0,88.0,46.5,77.800002,102.500000,126.00,208.0,2021,95.2,2021-12-28
362,12.0,29.0,88.0,46.0,77.300003,102.500000,123.50,197.0,2021,94.4,2021-12-29
363,12.0,30.0,88.0,48.0,80.500000,102.500000,124.25,182.0,2021,93.8,2021-12-30


In [33]:
ind1 = df['FLOW'] <= df['p_75']
ind2 = df['FLOW'] >= df['p_25']

In [42]:
ind3 = pd.DataFrame({'col1':ind1, 'col2':ind2}).all(axis=1)

In [43]:
df[ind1]

,MONTH,DAY,n,MIN,p_25,p_50,p_75,MAX,YEAR,FLOW,DATE
95,4.0,6.0,88.0,52.099998,99.075001,129.0,183.75,1020.0,2021,168.0,2021-04-06
96,4.0,7.0,88.0,52.099998,100.250000,133.5,199.25,1060.0,2021,174.0,2021-04-07
97,4.0,8.0,88.0,52.700001,103.000000,141.0,213.25,1080.0,2021,190.0,2021-04-08
98,4.0,9.0,88.0,50.099998,110.000000,147.0,245.25,1060.0,2021,208.0,2021-04-09
99,4.0,10.0,88.0,52.099998,112.750000,154.0,266.00,989.0,2021,231.0,2021-04-10
...,...,...,...,...,...,...,...,...,...,...,...
360,12.0,27.0,88.0,47.000000,78.200001,102.0,125.25,213.0,2021,95.9,2021-12-27
361,12.0,28.0,88.0,46.500000,77.800002,102.5,126.00,208.0,2021,95.2,2021-12-28
362,12.0,29.0,88.0,46.000000,77.300003,102.5,123.50,197.0,2021,94.4,2021-12-29
363,12.0,30.0,88.0,48.000000,80.500000,102.5,124.25,182.0,2021,93.8,2021-12-30


In [44]:
df[ind2]

,MONTH,DAY,n,MIN,p_25,p_50,p_75,MAX,YEAR,FLOW,DATE
0,1.0,1.0,88.0,51.0,81.499998,102.000000,122.00,170.0,2021,130.0,2021-01-01
1,1.0,2.0,88.0,51.0,80.275002,102.000000,120.25,173.0,2021,130.0,2021-01-02
2,1.0,3.0,88.0,50.5,78.424999,100.500000,119.00,182.0,2021,129.0,2021-01-03
3,1.0,4.0,88.0,50.0,77.699997,100.000000,119.50,180.0,2021,128.0,2021-01-04
4,1.0,5.0,88.0,51.0,76.799999,99.699997,116.75,191.0,2021,128.0,2021-01-05
...,...,...,...,...,...,...,...,...,...,...,...
360,12.0,27.0,88.0,47.0,78.200001,102.000000,125.25,213.0,2021,95.9,2021-12-27
361,12.0,28.0,88.0,46.5,77.800002,102.500000,126.00,208.0,2021,95.2,2021-12-28
362,12.0,29.0,88.0,46.0,77.300003,102.500000,123.50,197.0,2021,94.4,2021-12-29
363,12.0,30.0,88.0,48.0,80.500000,102.500000,124.25,182.0,2021,93.8,2021-12-30


In [45]:
df[ind3]

,MONTH,DAY,n,MIN,p_25,p_50,p_75,MAX,YEAR,FLOW,DATE
95,4.0,6.0,88.0,52.099998,99.075001,129.0,183.75,1020.0,2021,168.0,2021-04-06
96,4.0,7.0,88.0,52.099998,100.250000,133.5,199.25,1060.0,2021,174.0,2021-04-07
97,4.0,8.0,88.0,52.700001,103.000000,141.0,213.25,1080.0,2021,190.0,2021-04-08
98,4.0,9.0,88.0,50.099998,110.000000,147.0,245.25,1060.0,2021,208.0,2021-04-09
99,4.0,10.0,88.0,52.099998,112.750000,154.0,266.00,989.0,2021,231.0,2021-04-10
...,...,...,...,...,...,...,...,...,...,...,...
360,12.0,27.0,88.0,47.000000,78.200001,102.0,125.25,213.0,2021,95.9,2021-12-27
361,12.0,28.0,88.0,46.500000,77.800002,102.5,126.00,208.0,2021,95.2,2021-12-28
362,12.0,29.0,88.0,46.000000,77.300003,102.5,123.50,197.0,2021,94.4,2021-12-29
363,12.0,30.0,88.0,48.000000,80.500000,102.5,124.25,182.0,2021,93.8,2021-12-30
